<a href="https://colab.research.google.com/github/Vamsi-Malineni/Research-work/blob/master/pinn_trial_tf2_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp 
import os
import sys
import scipy.io
import time 


In [ ]:
class pinn:
  def __init__(self,layers,optimizer,ub,lb):
    
    self.model=tf.keras.Sequential()
    
    self.model.add(tf.keras.layers.InputLayer(input_shape=(layers[0],)))
    
    self.model.add(tf.keras.layers.Lambda(lambda X: 2.0*(X-lb)/(ub-lb)-1))

    for width in layers[1:]:
      self.model.add(tf.keras.layers.Dense(
          width,activation=tf.nn.tanh,
          kernel_initializer='glorot_normal'
      ))

  # finding the sizes of weights and biases for each layer in the neural network  
    self.sizes_w=[]
    self.sizes_b=[]

    for i,width in enumerate(layers):
      if i!=1:
        self.sizes_w.append(int(width*layers[1]))
        self.sizes_b.append(int(width if i!=0 else layers[1]))
    self.dtype=tf.float32

  # Defining two variables that have to be learnt (lambda1,lambda2)
    self.lambda1=tf.Variable([0.0],dtype=self.dtype)
    self.lambda2=tf.Variable([0.0],dtype=self.dtype)

    self.optimizer=optimizer

  def get_parameters(self,numpy=False):
    l1=self.lambda1
    l2=self.lambda2
    if numpy :
      return l1.numpy()[0],l2.numpy()[0]
    return l1,l2

  def setup_pinn(self,xtrain,ytrain,ttrain):
    l1,l2=self.get_parameters()
    # Inputs to be watched by gradient tape
    x=tf.convert_to_tensor(xtrain,dtype=self.dtype)
    y=tf.convert_to_tensor(ytrain,dtype=self.dtype)
    t=tf.convert_to_tensor(ttrain,dtype=self.dtype)
    
    with tf.GradientTape(persistent=True) as g:
      g.watch(x)
      g.watch(y)
      g.watch(t)
      
      

In [ ]:
def prepdata(N_train):
  data = scipy.io.loadmat('/content/drive/MyDrive/cylinder_nektar_wake.mat')
           
    U_star = data['U_star'] # N x 2 x T
    t_star = data['t'] # T x 1
    X_star = data['X_star'] # N x 2
    
    N = X_star.shape[0]
    T = t_star.shape[0]
    
    # Rearrange Data 
    XX = np.tile(X_star[:,0:1], (1,T)) # N x T
    YY = np.tile(X_star[:,1:2], (1,T)) # N x T
    TT = np.tile(t_star, (1,N)).T # N x T
    
    UU = U_star[:,0,:] # N x T
    VV = U_star[:,1,:] # N x T
    
    x = XX.flatten()[:,None] # NT x 1
    y = YY.flatten()[:,None] # NT x 1
    t = TT.flatten()[:,None] # NT x 1
    
    u = UU.flatten()[:,None] # NT x 1
    v = VV.flatten()[:,None] # NT x 1
    
    # Training Data    
    idx = np.random.choice(N*T, N_train, replace=False)
    x_train = x[idx,:]
    y_train = y[idx,:]
    t_train = t[idx,:]
    u_train = u[idx,:]
    v_train = v[idx,:]
    
    lb=tf.math.reduce_min(tf.concat([x_train,y_train,t_train],1),keepdims=True,axis=0) 
    ub=tf.math.reduce_max(tf.concat([x_train,y_train,t_train],1),keepdims=True,axis=0)
        

    return x_train,y_train,t_train,u_train,v_train,lb,ub
